Libraries

In [332]:
import warnings
warnings.filterwarnings('ignore')
import os
import glob
import librosa
import scipy.io.wavfile as wave
from scipy.fftpack import fft, ifft, fftshift
import matplotlib.pyplot as plt 
import numpy as np
from sklearn import mixture
import joblib
import pandas as pd
from sklearn.mixture import GaussianMixture
import json
from scipy.io import loadmat

Variables

In [333]:

n_mixtures = 7
max_iterations = 200
#calc_deltas=True
#sr=8000
#hop_length=int(0.005*sr)
home_path = os.getcwd()
#path = os.path.join(home_path, 'data')
print(home_path)

d:\CLG\S3-1\SSP\SSP-project


Extraction 

In [334]:
def extract_features(file_name):
    feature = loadmat('file_name')
    
    dur_list = np.array([[element for element in upperElement] for upperElement in feature['dur']])
    f0_list = np.array([[element for element in upperElement] for upperElement in feature['f0']])
    amp_tilt_list = np.array([[element for element in upperElement] for upperElement in feature['amp_tilt']])
    dur_tilt_list = np.array([[element for element in upperElement] for upperElement in feature['dur_tilt']])
    differ_list = np.array([[element for element in upperElement] for upperElement in feature['differ']])
    voiced_dur_list = np.array([[element for element in upperElement] for upperElement in feature['voiced_dur']])
    log_energy_list = np.array([[element for element in upperElement] for upperElement in feature['log_energy']])
    
    dur_list = dur_list.ravel()
    f0_list = f0_list.ravel()
    amp_tilt_list = amp_tilt_list.ravel()
    dur_tilt_list = dur_tilt_list.ravel()
    differ_list = differ_list.ravel()
    voiced_dur_list = voiced_dur_list.ravel()
    log_energy_list = log_energy_list.ravel()
    
    feature_vec = np.vstack((dur_list,f0_list,amp_tilt_list,dur_tilt_list,differ_list,voiced_dur_list,log_energy_list))
    return feature_vec
    #print(feature_vec)

Training Function

In [335]:
def training(train_data_path,feat_train_path,trained_model_path):
    all_speakers=glob.glob(train_data_path+'*')
    #print(all_speakers)
    print(2)

    directory=feat_train_path
    if not os.path.exists(directory):
        os.makedirs(directory)
      
    directory=trained_model_path
    if not os.path.exists(directory):
        os.makedirs(directory)

    print(len(all_speakers))
    for itr1 in range(0,len(all_speakers)):
        
        mats=glob.glob(all_speakers[itr1]+'/*.mat')
        spk=(all_speakers[itr1]).split("/")[-1]
        #print((mats))

        if not os.path.exists(directory):
            os.makedirs(directory)
      
        final_feat=np.empty([0, 35])
        
        for itr2 in range(0,len(mats)):
  
            #final_feat = eng.prosody(mats[itr2])
            #mat file
            mats[itr2]=mats[itr2].replace("\\","/")
            print(mats[itr2])
            final_feat = extract_features(mats[itr2])
            print(final_feat.shape)

        print(spk)    
        np.savetxt(feat_train_path+spk+"_all_features.txt", final_feat, delimiter=",")

        try:
            gmm = mixture.GaussianMixture(n_components=n_mixtures, covariance_type='diag' , max_iter = max_iterations ).fit(final_feat)
        except:
            print("ERROR : Error while training model for file "+spk)
          
        try:
            joblib.dump(gmm,trained_model_path+spk+'.pkl')
        except:
            print("ERROR : Error while saving model for "+spk)
          

    print("Training Completed")


Testing Function

In [336]:
def testing(test_data_path,feat_test,trained_model_path):
    # train feature extraction
    all_speakers=glob.glob(test_data_path+'*')

    import os
    directory=feat_test
    if not os.path.exists(directory):
        os.makedirs(directory)

    speakers = { all_speakers[k]:k for k in range(len(all_speakers)) }
    print(speakers) 
   
    num_test_cases={}
    tct={}
    for e in speakers:
         num_test_cases[e.replace(test_data_path,'')]=len(os.listdir(e))-1
         tct[e.replace(test_data_path,'')]=0

    # print(num_test_cases)

    spk_names = { all_speakers[k].replace(test_data_path,''):k for k in range(len(all_speakers)) }

    total_speakers=len(num_test_cases)

    confusion_matrix = np.zeros((total_speakers,total_speakers))


    for itr1 in range(0,len(all_speakers)):
        mats=glob.glob(all_speakers[itr1]+'/*.mat')
        spk=(all_speakers[itr1]).split("/")[-1]
        if not os.path.exists(directory):
            os.makedirs(directory)
        final_feat=np.empty([[]])
        output = {"Test_i":[],"Accent":[]}

        for itr2 in range(0,len(mats)):
            #print(wavs[itr2])

            feat = extract_features(mats[itr2])
            final_feat=np.concatenate((final_feat,feat),axis=0)
            #print(final_feat.shape)
            max_score=-np.inf
            max_spk_name=""

            for modelfile in sorted(glob.glob(trained_model_path+'*.pkl')):
                gmm = joblib.load(modelfile) 
                score=gmm.score(feat)
                #print score
                if score>max_score:
                    max_score,max_spk_name=score,modelfile.replace(trained_model_path,'').replace('.pkl','')

            print(spk+" -> "+max_spk_name+(" Y" if spk==max_spk_name  else " N"))
            output ["Test_i"].append(mats[itr2])
            output ["Accent"].append(max_spk_name)
            confusion_matrix[ spk_names[spk] ][spk_names[max_spk_name]]+=1
            tct[spk]+=1

        #print(spk)
        json_object = json.dumps(output, indent = 4)

        with open("Haram.json", "w") as outfile:
              outfile.write(json_object)
        np.savetxt(feat_test+spk+"_all_features.txt", feat, delimiter=",")
        
    return tct,confusion_matrix,total_speakers


File Paths

In [337]:
feat= 'D:/CLG/S3-1/SSP/SSP-project/feat/'
feat_train= 'D:/CLG/S3-1/SSP/SSP-project/feat/train/'
feat_test= 'D:/CLG/S3-1/SSP/SSP-project/feat/test/'
trained_model= 'D:/CLG/S3-1/SSP/SSP-project/trained_model/'
train_data= 'D:/CLG/S3-1/SSP/SSP-project/traindata/'
test_data= 'D:/CLG/S3-1/SSP/SSP-project/testdata/'

# for removing existing feature folders, models created
if os.path.exists('D:/CLG/S3-1/SSP/SSP-project/feat/'):
  !rm -rf  'D:/CLG/S3-1/SSP/SSP-project/feat/'
if os.path.exists('D:/CLG/S3-1/SSP/SSP-project/feat/test/'):
  !rm -rf  'D:/CLG/S3-1/SSP/SSP-project/feat/test/'

'rm' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.


Training

In [338]:
training(train_data,feat_train,trained_model)

2
3
D:/CLG/S3-1/SSP/SSP-project/traindata/Assam/assamf1.mat


FileNotFoundError: [Errno 2] No such file or directory: 'file_name.mat'

Testing

In [ ]:
tt,conf_mat,tot_spek=testing(test_data,feat_test,trained_model)

Accuracy

In [ ]:
print(tt)
print("Confusion Matrix:\n",conf_mat)
print("Total Speakers:",tot_spek)
print("Accuracy: ",(sum([ conf_mat[i][j] if i==j  else 0 for i in range(tot_spek) for j in range(tot_spek) ] )*100)/float(sum([i for i in tt.values()])))